In [9]:
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../../../")))
print(sys.path[-1])

/Users/seobi/PythonProjects/RAG_Commander/src


In [10]:
from tools.rag.document_loader.csv_loader import load_csv_loader
from utils.util import get_data_dir
path2024 = get_data_dir() / "population_insight" / "인구이동_전출입_2024년(연간) - 최종.csv"
path2025 = get_data_dir() / "population_insight" / "인구이동_전출입_2025년_1~8월_합산.csv"

In [11]:
documents2024 = load_csv_loader(path2024, encoding="utf-8-sig").load()
documents2025 = load_csv_loader(path2025, encoding="utf-8-sig").load()

In [12]:
total_docs = documents2024 + documents2025

In [13]:
len(total_docs)

24336

In [14]:
from langchain_community.vectorstores import PGVector 
from tools.rag.db_collection_name import MOVE_PEOPLE_KEY
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
load_dotenv()
POSTGRES_URL = os.getenv("POSTGRES_URL")
emb = OpenAIEmbeddings(model="text-embedding-3-large")
store = PGVector(
    collection_name=MOVE_PEOPLE_KEY,
    embedding_function=emb,
    connection_string=POSTGRES_URL,
)


/var/folders/fx/bdj1bhw11037xfwjzgrx6vc80000gn/T/ipykernel_78273/1301960732.py:8: LangChainPendingDeprecationWarning: Please use JSONB instead of JSON for metadata. This change will allow for more efficient querying that involves filtering based on metadata. Please note that filtering operators have been changed when using JSONB metadata to be prefixed with a $ sign to avoid name collisions with columns. If you're using an existing database, you will need to create a db migration for your metadata column to be JSONB and update your queries to use the new operators. 
  store = PGVector(


In [15]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from utils.llm import LLMProfile

system_prompt = """
당신은 TextToSQL 을 전문으로 맞을 LLM 입니다.

[스키마 설명]
테이블명: age_population
사용 DB: PostgresSQL
설명: 
  - 대한민국 지역 간 인구 이동 통계 데이터를 저장한 테이블입니다.
  - 전출지 혹은 전입지가 서울 및 서울의 자치구 지역만 모아놓은 데이터 입니다. 
  - 각 행(row)은 특정 연도(`year`)에 한 지역(`origin`)에서 다른 지역(`destination`)으로 이동한 인구 수(`total`)를 나타냅니다.
  - year가 2025 일 경우 해당 데이터는 1~8월 합쳐서 total을 만든것입니다.
  - year가 2024 일 경우 해당 데이터는 1~12월을 합쳐서 total을 만든것입니다.

컬럼 구조:
  - id: 정수형, 기본 키 (각 행의 고유 식별자)
  - year: 정수형, 데이터의 기준 연도 (예: 2024)
  - origin: 텍스트, 전출지 (예: "전국", "서울 영등포구")
  - destination: 텍스트, 전입지 (예: "서울", "서울 종로구")
  - total: 정수형, 해당 이동 인원 수 (단위: 명)

데이터 예시:
  | id | year | origin | destination | total  |
  |----|------|---------|-------------|-------|
  | 1  | 2024 | 전국    | 서울        | 893566 |
  | 2  | 2024 | 전국    | 서울 종로구 | 14483   |
  | 3  | 2024 | 전국    | 서울 중구   | 14333   |

주의사항:
  - `origin`과 `destination`은 모두 문자열이며, 시·도 또는 자치구 단위 명칭이 들어갑니다.
  - `total`은 특정 지역 쌍(origin → destination) 간의 이동 인원 총합을 나타냅니다.
  - 연도(`year`)별로 동일한 origin–destination 쌍이 존재할 수 있습니다.

[출력 형식]
- 사용자 질문에 맞는 PostgresSQL 쿼리문을 만들어주십시오
- 사용자 질문이 위 스키마 범위를 벗어나면 스키마에서 가능한 선까지만 쿼리를 생성하시오. 
- 반드시 SQL 코드만 출력하십시오. 어떤 형태의 설명, 따옴표, 백틱(`) 코드블록 마크다운(sql 등)도 포함하지 마십시오.
- 출력은 오직 순수한 SQL 문 한 줄 이상으로만 구성되어야 합니다.
"""

query = "2025년에 서ㅇ 동작구의 인구이동 어떻게돼??"

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", "{question}" )
])

chain = prompt | LLMProfile.dev_llm() | StrOutputParser()
ressult = chain.invoke({"question":query})




In [16]:
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
import os
load_dotenv()
connection_url = os.getenv("POSTGRES_URL")

engine = create_engine(connection_url)

with engine.connect() as conn:
    result = conn.execute(text(res))
    rows = [dict(row._mapping) for row in result.fetchall()]

print(rows)


NameError: name 'res' is not defined

In [ ]:
rows

[{'id': 12189,
  'year': 2025,
  'origin': '전국',
  'destination': '서울 동작구',
  'total': 29192},
 {'id': 12216,
  'year': 2025,
  'origin': '서울',
  'destination': '서울 동작구',
  'total': 14959},
 {'id': 12463,
  'year': 2025,
  'origin': '서울 종로구',
  'destination': '서울 동작구',
  'total': 151},
 {'id': 12710,
  'year': 2025,
  'origin': '서울 중구',
  'destination': '서울 동작구',
  'total': 124},
 {'id': 12957,
  'year': 2025,
  'origin': '서울 용산구',
  'destination': '서울 동작구',
  'total': 417},
 {'id': 13204,
  'year': 2025,
  'origin': '서울 성동구',
  'destination': '서울 동작구',
  'total': 249},
 {'id': 13451,
  'year': 2025,
  'origin': '서울 광진구',
  'destination': '서울 동작구',
  'total': 300},
 {'id': 13698,
  'year': 2025,
  'origin': '서울 동대문구',
  'destination': '서울 동작구',
  'total': 287},
 {'id': 13945,
  'year': 2025,
  'origin': '서울 중랑구',
  'destination': '서울 동작구',
  'total': 186},
 {'id': 14192,
  'year': 2025,
  'origin': '서울 성북구',
  'destination': '서울 동작구',
  'total': 312},
 {'id': 14439,
  'year': 2025,
  '